In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#单组热成像数据为24x24的温度值
img_height_width = 24                       

#取数据集前650组作为样本，串口波特率921600，采样时间大约需要75s
dataset_row = 650                           
dataset_col = np.square(img_height_width)

#分类的种类
category_num = 4

#600组作为训练集，50组作为测试集
train_data_len = 600
test_data_len = 50

random_seed = 100

In [ ]:
#数据从SecureCRT保存的log信息中获得
dataset = np.zeros((category_num,dataset_row,dataset_col))

#识别调节滚动条手势为5分类 
if category_num==5 :
    dataset[0] = np.genfromtxt("dataset\\none.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[1] = np.genfromtxt("dataset\\open.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[2] = np.genfromtxt("dataset\\close_far.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[3] = np.genfromtxt("dataset\\cross.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[4] = np.genfromtxt("dataset\\close_near.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]

#识别数字手势为4分类
else:
    dataset[0] = np.genfromtxt("dataset\\none.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[1] = np.genfromtxt("dataset\\1.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[2] = np.genfromtxt("dataset\\2.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    dataset[3] = np.genfromtxt("dataset\\3.log", delimiter=",", dtype='float32',encoding='utf-8')[1:dataset_row+1]
    
#[1:dataset_row+1]为了避免第一组数据乱码

In [ ]:
np.random.seed(random_seed)
for i in range(category_num):
    np.random.shuffle(dataset[i])

In [ ]:
train_data = np.zeros((category_num,train_data_len,dataset_col))
test_data = np.zeros((category_num,test_data_len,dataset_col))

for i in range(category_num):
    train_data[i] = dataset[i][0:train_data_len]
    test_data[i] = dataset[i][train_data_len:train_data_len+test_data_len]

train_data.shape,test_data.shape

In [ ]:
# graph = train_data[3][200].reshape(24, 24)
# plt.imshow(graph, cmap=plt.cm.Reds)
# plt.colorbar()
# plt.show()

In [ ]:
for i in range(category_num):
    train_data[i] -= np.average(train_data[i],axis = 1).reshape(-1,1)
    test_data[i] -= np.average(test_data[i],axis = 1).reshape(-1,1)
train_data /= 10
test_data /= 10

In [ ]:
#train_data[x][y] 可视化第x类的第y组数据
graph = train_data[3][25].reshape(24, 24)
plt.imshow(graph, cmap=plt.cm.Reds)
plt.colorbar()
plt.show()

In [ ]:
train_data = train_data.reshape(-1,dataset_col)
train_data = train_data.reshape(train_data.shape[0],24,24,1)

test_data = test_data.reshape(-1,dataset_col)
test_data = test_data.reshape(test_data.shape[0],24,24,1)

train_data.shape,test_data.shape

In [ ]:
train_label = np.zeros(train_data.shape[0])
test_label = np.zeros(test_data.shape[0])

#每一类的数据赋予标签值
base = 0
for i in range(category_num):
    train_label[base:base+train_data_len] = i
    base += train_data_len
    
base = 0
for i in range(category_num):
    test_label[base:base+test_data_len] = i
    base += test_data_len

#转为One-hot
train_label = to_categorical(train_label)
test_label = to_categorical(test_label)

In [ ]:
network = Sequential()
network.add(Conv2D(5, (3,3),input_shape=(img_height_width, img_height_width,1),activation='relu'))
network.add(MaxPooling2D(pool_size=(2,2),padding='valid'))
network.add(Flatten())
network.add(Dense(128, activation='relu'))
network.add(Dense(64, activation='relu'))
network.add(Dense(category_num, activation='softmax'))
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

network.fit(train_data, train_label, validation_split = 0.2, epochs=5, batch_size=10)
network.save("thermo_gesture.h5")

test_loss, test_acc = network.evaluate(test_data, test_label)
print("test_loss:", test_loss)
print("test_acc:", test_acc)

In [ ]:
#TensorFlow Lite模型量化

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(network)
tflite_model = converter.convert()
# Save the model to disk
open("model.tflite", "wb").write(tflite_model)


# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(network)
# Indicate that we want to perform the default optimizations,
# which includes quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]


# Define a generator function that provides our test data's x values
# as a representative dataset, and tell the converter to use it
def representative_dataset_generator():
    for value in test_data:
    # Each scalar value must be inside of a 2D array that is wrapped in a list
        yield [np.array(value, dtype=np.float32, ndmin=4)]


converter.representative_dataset = representative_dataset_generator
# Convert the model
tflite_model = converter.convert()
open("model_quantized.tflite", "wb").write(tflite_model)

In [ ]:
# 使用Linux Shell 命令xxd将tflite文件转换为数组
# xxd -i model_quantized.tflite > model.cpp